# 한영 기계번역

## 개요

- 사용 데이터: 일상 생활 및 구어체 한영 번역 데이터

- 설계
| 태스크 | 기술 |
|----------------|-----------------------------|
| 데이터 전처리 | 데이터 로드, 전처리 |
| 토큰화 | BPE (Byte Pair Encoding) |
| 사용 모델 | Seq2Seq (GRU), Seq2Seq with Attention (GRU)  |
| 사용 어텐션 | Scaled Luong Attention |

# 1. 데이터 전처리 및 토큰화

In [ ]:
!pip install sentencepiece

In [ ]:
# 파일 경로 및 파일 읽기 라이브러리
from pathlib import Path
from dataclasses import dataclass
import math
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from typing import Dict, List, Any, Tuple, Optional

# 토큰 관련 라이브러리
import sentencepiece as spm

# 파이토치 관련 라이브러리
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

# 평가용
import sacrebleu

# 훈련 시 시각화
from tqdm import tqdm

In [ ]:
"""
Config 클래스 정의

- 상수 변수 정의
- @dataclass 데코레이터 추가
"""
@dataclass
class Config:
    # 데이터 폴더 생성
    root = Path(".")
    raw_dir = root / "data"
    data_dir = raw_dir / "mission11_koen"
    model_dir = root / "models"

    # 토크나이저용 변수
    vocab_size = 4000
    coverage = 1.0

    # 데이터로더용 변수 선언
    max_len = 100
    batch_size = 32
    num_workers = 2

    # 데이터 분리용 변수 선언
    seed = 42

    # 모델용 변수 선언
    emb_dim = 256
    hid_dim = 512
    num_layers = 1
    dropout = 0.1

    # 학습용 변수 선언: 학습률, 에포크, 인내심, 최소 개선 폭
    lr = 3e-4
    epochs = 10
    patience = 3
    min_delta = 0.01
    teacher_forcing = 0.5
    grad_clip = 1.0

    # 평가용 변

    # 디바이스 설정
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# 변수 생성 및 폴더 생성
cfg = Config()

# 모델, 데이터 저장 폴더
cfg.model_dir.mkdir(parents=True, exist_ok=True)
cfg.raw_dir.mkdir(parents=True, exist_ok=True)

## 2. 데이터 다운로드

In [ ]:
"""
데이터 다운로드

- 미션 페이지에서 데이터 다운로드
"""
with zipfile.ZipFile(f"{str(cfg.raw_dir)}/mission11_koen.zip") as z:
    z.extractall(path=f"{str(cfg.raw_dir)}/mission11_koen")

In [ ]:
# JSON 파일 로드 함수
def load_json(json_path: Path):
    with open(str(json_path), "r", encoding="utf-8") as j:
        data = json.load(j)
    return data

In [ ]:
# 훈련, 검증 데이터 불러오기
train_data = load_json(cfg.data_dir / "일상생활및구어체_한영_train_set.json")
val_data = load_json(cfg.data_dir / "일상생활및구어체_한영_valid_set.json")

In [ ]:
# 훈련 데이터 샘플 출력
print(train_data["data"][0])
print(f"원문: {train_data["data"][0].get("ko")}")
print(f"번역: {train_data["data"][0].get("en")}")
print(type(train_data))

{'sn': 'INTSALDSUT062119042703238', 'data_set': '일상생활및구어체', 'domain': '해외영업', 'subdomain': '도소매유통', 'ko_original': '원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.', 'ko': '원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.', 'mt': 'If you reply to the color you want, we will start making it right away.', 'en': 'If you reply to the color you want, we will start making it right away.', 'source_language': 'ko', 'target_language': 'en', 'word_count_ko': 7, 'word_count_en': 15, 'word_ratio': 2.143, 'file_name': 'INTSAL_DSUT.xlsx', 'source': '크라우드소싱', 'license': 'open', 'style': '구어체', 'included_unknown_words': False, 'ner': None}
원문: 원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.
번역: If you reply to the color you want, we will start making it right away.
<class 'dict'>


In [ ]:
# 데이터 정규화 함수 생성
def extract_pairs(json_data: dict):
    """
    데이터셋을 "list[{'ko':..., 'en':...}]" 형태로 정규화(페어 추출)

    arg:
        dict: json 데이터

    반환:
        pairs: 각 원소는 {'ko': str, 'en': str}
    """
    if isinstance(json_data, dict):
        pairs_list = []
        for d in json_data["data"]:
            kor = d.get("ko")
            eng = d.get("en")
            pairs_list.append({"ko": kor, "en": eng})

    else:
        raise TypeError(f"Unsupported JSON root type: {type(json_data)}")


    # 검증
    normalized_pairs: List[Dict[str, str]] = []
    for item in pairs_list:

        # 불량 데이터 거르기
        ko_text = item.get("ko", None)
        en_text = item.get("en", None)

        if ko_text is None or en_text is None:
            continue

        # 문자열이 아니면 강제로 변환
        normalized_pairs.append({"ko": str(ko_text), "en": str(en_text)})

    return normalized_pairs

In [ ]:
# json 데이터 정규화
train_normalized = extract_pairs(train_data)
val_normalized = extract_pairs(val_data)

In [ ]:
# 정규화 데이터 예시 출력
train_normalized[0]

{'ko': '원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.',
 'en': 'If you reply to the color you want, we will start making it right away.'}

## 3. BPE 토크나이저

In [ ]:
# bpe 토크나이저 훈련 함수
def train_bpe(
        texts: List[str],
        model_prefix: str,
        model_dir: Path=cfg.model_dir,
        vocab_size: int=cfg.vocab_size,
        char_coverage: float=cfg.coverage
        ) -> Path:
    """
    BPE 모델 학습 후 model_path 반환

    args:
        - text: 학습에 사용할 문장 리스트
        - model_prefix: 출력 모델의 prefix
        - model_dir: 모델 저장 경로

    returns:
        - model_path
    """
    model_dir.mkdir(parents=True, exist_ok=True)

    # 훈련용 임시 텍스트 파일 생성
    train_txt_path = model_dir / f"{model_prefix}.txt"
    with open(train_txt_path, "w", encoding="utf-8") as f:
        for line in texts:
            # 줄 단위로 기록 (SentencePiece는 한 줄을 힌 문장으로 인식)
            f.write(line.replace("\n", " ").strip() + "\n")

    # special token ids 지정:
    # pad_id=0, bos_id=1, eos_id=2, unk_id=3로 고정
    spm.SentencePieceTrainer.train(
        input=str(train_txt_path),
        model_prefix=str(model_dir / model_prefix),
        vocab_size=vocab_size,
        model_type="bpe",  # BPE 사용
        character_coverage=char_coverage,

        # 특수 토큰 지정
        pad_id=0,
        bos_id=1,
        eos_id=2,
        unk_id=3,

        # 학습 안정성/일반성 옵션
        normalization_rule_name="nmt_nfkc",
    )

    model_path = model_dir / f"{model_prefix}.model"
    return model_path

In [ ]:
# 토크나이저 래퍼 생성
class SentencePieceTokenizer:
    """
    토크나이저 래퍼

    - encode: 텍스트 -> 토큰 id
    - decode: 토큰 id -> 텍스트
    """
    def __init__(self, model_path: Path):
        self.tokenizer = spm.SentencePieceProcessor()
        self.tokenizer.load(str(model_path))

        # 특수 토큰 id (정수)
        self.pad_id = self.tokenizer.pad_id()
        self.unk_id = self.tokenizer.unk_id()
        self.bos_id = self.tokenizer.bos_id()
        self.eos_id = self.tokenizer.eos_id()

        # 안전성 체크
        assert self.pad_id >= 0
        assert self.bos_id >= 0
        assert self.eos_id >= 0

    def encode(self, text_str: str, max_len: int) -> List[int]:
        sub_ids = self.tokenizer.encode(text_str, out_type=int)

        max_content_len = max_len - 2
        sub_ids = sub_ids[:max_content_len]

        return [self.bos_id] + sub_ids + [self.eos_id]

    def decode(self, token_ids: List[int]) -> str:
        filtered_ids = [
            tid for tid in token_ids
            if tid not in (self.bos_id, self.eos_id, self.pad_id)
        ]
        return self.tokenizer.decode(filtered_ids)

In [ ]:
"""
bpe 토크나이저 학습

- 한국어, 영어 언어별 토그나이저 훈련 및 생성
- 생성 파일
    bpe_ko.model
    bpe_ko.vocab
    bpe_en.model
    bpe_en.vocab
"""
# 훈련 데이터에서 언어별 분리
ko_text = [p["ko"] for p in train_normalized]
en_text = [p["en"] for p in train_normalized]

# 한국어 BPE 학습
ko_bpe_path = train_bpe(
    texts=ko_text,
    model_prefix="bpe_ko",
)

# 영어 BPE 학습
en_bpe_path = train_bpe(
    texts=en_text,
    model_prefix="bpe_en"
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: models/bpe_ko.txt
  input_format: 
  model_prefix: models/bpe_ko
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  diffe

In [ ]:
# 생성 토크나이저 로드
ko_tok = SentencePieceTokenizer(ko_bpe_path)
en_tok = SentencePieceTokenizer(en_bpe_path)

# 토크나이저 동작 확인
print(ko_tok.encode("안녕하세요.", 20))
print(en_tok.encode("Heollo, friends", 20))

[1, 350, 1831, 2]
[1, 383, 3798, 560, 3817, 2611, 2]


# 4. 데이터셋/데이터로더

## 4.1. 데이터셋

In [ ]:
# 데이터셋 클래스 정의
class TranslationDataset(Dataset):
    """
    번역 데이터셋 클래스

    args:
        - pairs: 한영 번역 데이터 리스트
        - src_tok: 소스언어 토크나이저
        - tgt_tok: 타겟언어 토크나이저
        - max_len_src: 소스 언어 인코딩 최대 허용 길이
        - max_len_tgt: 타겟 언어 인코딩 최대 허용 길이

    returns:
        - dataset: 튜플로 감싸진 한영 데이터셋
    """
    def __init__(
            self,
            pairs: List[Dict[str, str]],
            src_tok: SentencePieceTokenizer,
            tgt_tok: SentencePieceTokenizer,
            max_len_src: int=cfg.max_len,
            max_len_tgt: int=cfg.max_len,
            ):

        self.pairs = pairs
        self.src_tok = src_tok
        self.tgt_tok = tgt_tok
        self.max_len_src = max_len_src
        self.max_len_tgt = max_len_tgt

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        src_ids = self.src_tok.encode(item["ko"], self.max_len_src)
        tgt_ids = self.tgt_tok.encode(item["en"], self.max_len_tgt)
        return src_ids, tgt_ids

In [ ]:
# 훈련/검증 데이터셋 생선
train_ds = TranslationDataset(train_normalized, ko_tok, en_tok)
val_ds = TranslationDataset(val_normalized, ko_tok, en_tok)

## 4.2. 데이터로더(with Collate_fn)

In [ ]:
# 커스텀 collate_fn 함수 생성
def collate_fn(pad_id_src: int, pad_id_tgt: int):
    def _fn(batch):
        src, tgt = zip(*batch)
        src_len = max(len(x) for x in src)
        tgt_len = max(len(x) for x in tgt)

        src_tensor = torch.full((len(src), src_len), pad_id_src, dtype=torch.long)
        tgt_tensor = torch.full((len(tgt), tgt_len), pad_id_tgt, dtype=torch.long)

        for idx, ko in enumerate(src):
            src_tensor[idx, :len(ko)] = torch.tensor(ko)
        for idx, en in enumerate(tgt):
            tgt_tensor[idx, :len(en)] = torch.tensor(en)

        return src_tensor, tgt_tensor

    return _fn


In [ ]:
# 훈련/검증 데이터로더 생성
train_loader = DataLoader(
    train_ds,
    batch_size=cfg.batch_size,
    shuffle=True,
    num_workers=cfg.num_workers,
    collate_fn=collate_fn(ko_tok.pad_id, en_tok.pad_id)
)

val_loader = DataLoader(
    val_ds,
    batch_size=cfg.batch_size,
    shuffle=False,
    num_workers=cfg.num_workers,
    collate_fn=collate_fn(ko_tok.pad_id, en_tok.pad_id)
)

In [ ]:
# 샘플 확인
src_sam, tgt_sam = next(iter(train_loader))

print(src_sam)
print(tgt_sam)

tensor([[   1,    6, 1901,  ...,    0,    0,    0],
        [   1,  413,  341,  ...,    0,    0,    0],
        [   1,    6,   53,  ...,    0,    0,    0],
        ...,
        [   1,  220, 1888,  ...,    0,    0,    0],
        [   1,  414, 1472,  ...,    0,    0,    0],
        [   1,   94, 1972,  ...,    0,    0,    0]])
tensor([[   1,   38,  540,  ...,    0,    0,    0],
        [   1,  489,  700,  ...,    0,    0,    0],
        [   1,   38, 1088,  ...,    0,    0,    0],
        ...,
        [   1,  233,   30,  ...,    0,    0,    0],
        [   1, 2215, 3817,  ...,    0,    0,    0],
        [   1, 1162,   30,  ...,    0,    0,    0]])


# 5. 모델링

## 5.1. Seq2Seq

In [ ]:
"""
Encoder 정의

입력 문장(한국어)을 읽어서
문맥 표현(hidden states)을 생성하는 역할
"""
class Encoder(nn.Module):
    def __init__(self, vocab, emb, hid, pad_id):
        """
        args:
            - vocab (int): 입력 언어 vocabulary 크기 (한국어 BPE vocab size)
            - emb (int): 임베딩 차원 (각 토큰을 몇 차원 벡터로 표현할지)
            - hid (int): GRU 은닉 상태 차원
            - pad_id (int): PAD 토큰 id (embedding에서 무시하기 위해 필요)
        """
        super().__init__()

        # Embedding layer
        # 입력: (batch, seq_len) -> 토큰 ID
        # 출력: (batch, seq_len, emb) -> 임베딩 벡터
        self.emb = nn.Embedding(
            num_embeddings=vocab,
            embedding_dim=emb,
            padding_idx=pad_id
        )

        # GRU
        # batch_first=True -> 입력/출력 텐서 shape를 (B, T, D)로 통일
        self.gru = nn.GRU(
            input_size=emb,
            hidden_size=hid,
            batch_first=True
        )

    def forward(self, x):
        """
        x: (batch_size, src_seq_len) 한국어 문장이 토큰 id로 변환된 상태
        """
        #  토큰 id -> 임베딩 벡터
        # (B, T) -> (B, T, emb)
        x = self.emb(x)

        # GRU 통과
        # outputs: (B, T, hid)
        # - 모든 타임스텝의 은닉 상태 (attention에서 사용)
        # hidden: (1, B, hid)
        # - 마지막 타임스텝 은닉 상태 (decoder 초기 상태)
        outputs, hidden = self.gru(x)

        return outputs, hidden


In [ ]:
"""
Decoder 정의

이전 토큰과 hidden state를 받아
다음 단어(영어)를 하나 생성
"""
class Decoder(nn.Module):
    def __init__(self, vocab, emb, hid, pad_id):
        """
        args:
            vocab (int): 출력 언어 vocabulary 크기 (영어 BPE vocab size)
            emb (int): 임베딩 차원
            hid (int): GRU 은닉 상태 차원
            pad_id (int): PAD 토큰 id
        """
        super().__init__()

        # 출력 토큰용 embedding
        self.emb = nn.Embedding(
            num_embeddings=vocab,
            embedding_dim=emb,
            padding_idx=pad_id
        )

        # GRU
        # 입력은 "현재 단어 임베딩 1개"
        self.gru = nn.GRU(
            input_size=emb,
            hidden_size=hid,
            batch_first=True
        )

        # Linear layer
        # GRU hidden -> vocabulary 크기의 logits으로 선형변환
        self.fc = nn.Linear(hid, vocab)

    def forward(self, x, hidden):
        """
        x: (batch_size,) 이전 시점의 토큰 id (영어 단어 1개)

        hidden: (1, batch_size, hid) 이전 시점의 GRU 은닉 상태
        """

        # 단어 id -> embedding
        # unsqueeze(1): (B,) → (B, 1)
        # embedding 후: (B, 1, emb)
        x = self.emb(x.unsqueeze(1))

        # GRU 한 스텝 실행
        # out: (B, 1, hid)
        # hidden: (1, B, hid)
        out, hidden = self.gru(x, hidden)

        # 단어 예측 점수(logits) 생성
        # out.squeeze(1): (B, hid)
        # logits: (B, vocab)
        logits = self.fc(out.squeeze(1))

        return logits, hidden


In [ ]:
"""
Seq2Seq 정의

- Encoder + Decoder를 묶은 전체 번역 모델
"""
class Seq2Seq(nn.Module):
    def __init__(self, enc, dec, bos_id, device=cfg.device):
        """
        args:
            - enc : Encoder 객체
            - dec : Decoder 객체
            - bos_id : <BOS> 토큰 id (문장 시작)
            - device : cpu / cuda
        """
        super().__init__()
        self.enc = enc
        self.dec = dec
        self.bos_id = bos_id
        self.device = device

    def forward(self, src, tgt, teacher_forcing: float = cfg.teacher_forcing):
        """
        args:
            - src: (B, src_len) 한국어 문장 토큰 id 시퀀스

            - tgt: (B, tgt_len) 영어 문장 토큰 id 시퀀스
                 학습 시에는 정답 문장 전체가 들어옴

            - teacher_forcing: 정답 단어를 다음 입력으로 쓸 확률
        """

        # 기본 정보 추출
        B, T = tgt.size()  # batch_size, target sequence length
        vocab = self.dec.fc.out_features  # 영어 vocab 크기

        # 전체 시점의 출력을 저장할 텐서
        # shape: (B, T, vocab)
        logits_all = torch.zeros(B, T, vocab, device=self.device)

        # Encoder 실행
        # enc_out: (B, src_len, hid)
        # hidden : (1, B, hid)
        enc_out, hidden = self.enc(src)

        # 디코더 첫 입력은 <BOS>
        # tgt[:, 0] == BOS
        x = tgt[:, 0]

        # Decoder 타임스텝 반복
        for t in range(1, T):
            # 한 단어 예측 (hidden은 인코더 유래)
            logits, hidden = self.dec(x, hidden)

            # 현재 시점 예측 결과 저장
            logits_all[:, t] = logits

            # Teacher Forcing 여부 결정
            use_tf = torch.rand(1).item() < teacher_forcing

            # 다음 입력 결정
            # - teacher forcing: 정답 단어
            # - 아니면: 모델이 예측한 단어
            x = tgt[:, t] if use_tf else logits.argmax(dim=1)

        return logits_all


In [ ]:
# 모델 생성
enc = Encoder(
    vocab=ko_tok.tokenizer.get_piece_size(),
    emb=cfg.emb_dim,
    hid=cfg.hid_dim,
    pad_id=ko_tok.pad_id,
)

dec = Decoder(
    vocab=en_tok.tokenizer.get_piece_size(),
    emb=cfg.emb_dim,
    hid=cfg.hid_dim,
    pad_id=en_tok.pad_id,
)

model_s2s = Seq2Seq(enc, dec, bos_id=en_tok.bos_id).to(cfg.device)

## 5.2. Attention Seq2Seq

In [ ]:
"""
Scaled Luong Attention (dot-product attention + scaling) 설계

- 디코더의 현재 상태(query)가 인코더 출력(keys/values) 중 어디에 가장 집중해야 하는지를 계산
- Scaled Luong attention에서는 별도의 Query, Key, Value를 선형층으로 만들지 않고 GRU의 은닉 상태를 그대로 사용한다.
"""
class ScaledLuongAttention(nn.Module):
    def __init__(self, hid_dim):
        """
        args:
            - hid_dim (int): encoder / decoder GRU의 은닉 차원. dot-product의 분산을 안정화하기 위한 스케일링에 사용
        """
        super().__init__()

        # 제곱근 스케일링을 통해 dot(query, key)의 값이 hid_dim에 비례해 커지는 것을 방지
        self.scale = math.sqrt(hid_dim)

    def forward(self, query, keys, values, mask=None):
        """
        query:
            shape = (B, hid) 디코더의 현재 hidden state

        keys:
            shape = (B, T_src, hid) 인코더의 모든 타임스텝 hidden states

        values:
            shape = (B, T_src, hid) 실제로 가중합에 사용될 벡터
            Luong에서는 keys == values

        mask:
            shape = (B, T_src)
            PAD 토큰 위치를 무시하기 위한 마스크
        """

        # Attention score 계산 (dot-product)
        # query.unsqueeze(2): (B, hid) -> (B, hid, 1)
        # keys:               (B, T_src, hid)
        #
        # torch.bmm:
        # (B, T_src, hid) @ (B, hid, 1)
        # (B, T_src, 1)
        scores = torch.bmm(keys, query.unsqueeze(2)).squeeze(2)   # (B, T_src)

        # Scaling
        # dot-product 값이 너무 커지는 것을 방지
        # softmax가 과도하게 한 위치만 선택하는 문제 완화
        scores = scores / self.scale

        # PAD 위치 마스킹
        # mask == 0 인 위치는 PAD
        # 해당 위치 score를 매우 작은 값으로 설정
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # Attention weight 계산
        attn = torch.softmax(scores, dim=1)  # (B, T_src)

        # Context vector 계산 (가중합)
        # attn.unsqueeze(1): (B, 1, T_src)
        # values:            (B, T_src, hid)
        #
        # 결과: (B, 1, hid) -> squeeze -> (B, hid)
        context = torch.bmm(attn.unsqueeze(1), values).squeeze(1)

        return context


In [ ]:
# 어텐션 디코더
class AttnDecoder(nn.Module):
    """
    Attention을 사용하는 디코더

    구조:
    - 이전 단어 embedding
    - attention으로 얻은 context
    - embddding과 context를 concat
    - GRU
    - 다음 단어 예측
    """
    def __init__(self, vocab, emb, hid, pad_id):
        super().__init__()

        # 출력 토큰 embedding
        self.emb = nn.Embedding(
            num_embeddings=vocab,
            embedding_dim=emb,
            padding_idx=pad_id
        )

        # Attention 모듈
        self.attn = ScaledLuongAttention(hid)

        # GRU
        # 입력 차원 = (embedding + context)
        # context는 encoder hidden(hid)
        self.gru = nn.GRU(
            input_size=emb + hid,
            hidden_size=hid,
            batch_first=True
        )

        # 출력 projection
        # GRU 출력 + context를 결합해 vocab logits 생성
        self.fc = nn.Linear(hid * 2, vocab)

    def forward(self, x, hidden, enc_out, mask):
        """
        args:
            - x: shape = (B,) 이전 시점의 출력 토큰 id
            - hidden: shape = (1, B, hid) 이전 시점의 디코더 hidden state
            - enc_out: shape = (B, T_src, hid) 인코더의 모든 타임스텝 출력
            - mask: shape = (B, T_src) PAD 위치 마스크
        """

        # 이전 단어 embedding
        # (B,) -> (B, 1) -> (B, 1, emb)
        # GRU는 (batch, time, feature) 형태의 입력을 기대하므로 시간축(unsqueeze(1))을 추가
        emb = self.emb(x.unsqueeze(1))

        # Attention 계산
        # hidden[-1]:
        #   (1, B, hid) -> (B, hid)
        #   마지막 레이어의 현재 hidden state
        context = self.attn(
            hidden[-1],   # query
            enc_out,      # keys
            enc_out,      # values
            mask
        )  # (B, hid)

        # GRU 입력 구성
        # emb:     (B, 1, emb)
        # context: (B, hid) -> (B, 1, hid)
        gru_in = torch.cat(
            [emb, context.unsqueeze(1)],
            dim=2
        )  # -> (B, 1, emb + hid)

        # GRU 한 스텝 실행
        out, hidden = self.gru(gru_in, hidden)
        # out:    (B, 1, hid)
        # hidden: (1, B, hid)

        # 단어 예측
        # out.squeeze(1): (B, hid)
        # context:        (B, hid)
        logits = self.fc(
            torch.cat([out.squeeze(1), context], dim=1)
        )  # -> (B, vocab)

        return logits, hidden


In [ ]:
"""
Encoder + Attention Decoder를 결합한 전체 번역 모델
"""
class Seq2SeqWithAttention(nn.Module):
    def __init__(self, enc, dec, bos_id, pad_id_src, device=cfg.device):
        super().__init__()
        self.enc = enc
        self.dec = dec
        self.bos_id = bos_id
        self.pad_id_src = pad_id_src
        self.device = device

    def forward(self, src, tgt, teacher_forcing: float=cfg.teacher_forcing):
        """
        args:
            - src: shape = (B, T_src)
            - tgt: shape = (B, T_tgt)
            - teacher_forcing: 다음 입력으로 정답을 쓸 확률
        """

        # 기본 정보
        B, T = tgt.size()
        vocab = self.dec.fc.out_features

        # 전체 타임스텝 출력 저장용
        logits_all = torch.zeros(
            B, T, vocab,
            device=self.device
        )

        # Encoder 실행
        enc_out, hidden = self.enc(src)
        # enc_out: (B, T_src, hid)
        # hidden:  (1, B, hid)

        # PAD 마스크 생성
        mask = (src != self.pad_id_src)

        # 디코더 시작 입력: <BOS>
        x = tgt[:, 0]

        # 디코딩 루프
        for t in range(1, T):
            logits, hidden = self.dec(
                x,
                hidden,
                enc_out,
                mask
            )

            logits_all[:, t] = logits

            # teacher forcing 여부 결정
            use_tf = torch.rand(1).item() < teacher_forcing

            # 다음 입력 선택
            x = tgt[:, t] if use_tf else logits.argmax(dim=1)

        return logits_all


In [ ]:
# 모델 생성
enc_attn = Encoder(
    vocab=ko_tok.tokenizer.get_piece_size(),
    emb=cfg.emb_dim,
    hid=cfg.hid_dim,
    pad_id=ko_tok.pad_id,
)

dec_attn = AttnDecoder(
    vocab=en_tok.tokenizer.get_piece_size(),
    emb=cfg.emb_dim,
    hid=cfg.hid_dim,
    pad_id=en_tok.pad_id,
)

model_attn = Seq2SeqWithAttention(
    enc_attn,
    dec_attn,
    bos_id=en_tok.bos_id,
    pad_id_src=ko_tok.pad_id,
).to(cfg.device)

# 6. 훈련

## 6.1. 유틸

In [ ]:
# 조기 종료 클래스
class EarlyStopping:
    def __init__(self, patience: int=cfg.patience, min_delta: float = cfg.min_delta, save_path: str | Path = cfg.model_dir / "best_model.pt"):
        """
        Args:
            patience (int): 개선이 없을 때 허용 epoch 수
            min_delta (float): 최소 개선 폭
            save_path (str | Path): best model 저장 경로
        """
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float("inf")
        self.counter = 0
        self.save_path = Path(save_path)

    def step(self, val_loss: float, model: torch.nn.Module) -> bool:
        """
        Args:
            val_loss (float): 현재 epoch의 validation loss
            model (nn.Module): 현재 모델

        Returns:
            bool: True면 학습 중단, False면 계속
        """
        if val_loss < self.best_loss - self.min_delta:
            # 성능 개선
            self.best_loss = val_loss
            self.counter = 0

            # best model 저장
            torch.save(
                {"model_state": model.state_dict()},
                self.save_path
            )
            print(f"Validation loss improved. Best model saved to {self.save_path}")

            return False
        else:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} / {self.patience}")

            return self.counter >= self.patience

In [ ]:
# 훈련 함수
def my_trainer(model, loader, optim, loss_fn, grad_clip: float=cfg.grad_clip, device=cfg.device):
    model.train()
    total = 0

    for src, tgt in loader:
        src, tgt = src.to(device), tgt.to(device)
        optim.zero_grad()

        # logits shape: (B, T, V)
        logits = model(src, tgt)

        # CrossEntropyLoss를 위한 입력으로 input : (N, C), target: (N,)로 변환
        loss = loss_fn(
            logits[:, 1:].reshape(-1, logits.size(-1)),
            tgt[:, 1:].reshape(-1),
        )

        loss.backward()

        # gradient clipping (폭주 방지)
        nn.utils.clip_grad_norm_(
            model.parameters(),
            max_norm=grad_clip
        )

        optim.step()
        total += loss.item()

    return total / len(loader)

In [ ]:
# 검증 함수
def my_evaluate(model, loader, loss_fn, device=cfg.device):
    model.eval()
    total = 0

    with torch.no_grad():
        for src, tgt in loader:
            src, tgt = src.to(device), tgt.to(device)

            # logits shape: (B, T, V)
            logits = model(src, tgt)

            # CrossEntropyLoss를 위한 입력으로 input : (N, C), target: (N,)로 변환
            loss = loss_fn(
                logits[:, 1:].reshape(-1, logits.size(-1)),
                tgt[:, 1:].reshape(-1),
            )

            total += loss.item()

    return total / len(loader)

## 6.2. 훈련 루프 실행

In [ ]:
# 손실함수, 최적화 알고리즘
optimizer_s2s = torch.optim.Adam(model_s2s.parameters(), lr=cfg.lr)
optimizer_attn = torch.optim.Adam(model_attn.parameters(), lr=cfg.lr)
loss_fn = nn.CrossEntropyLoss(ignore_index=en_tok.pad_id)

# 조기종료 인스턴스
s2s_early_stopping = EarlyStopping(save_path=cfg.model_dir / "s2s_model.pt")
attn_early_stopping = EarlyStopping(save_path=cfg.model_dir / "attn_model.pt")

In [ ]:
# s2s 모델 훈련
for epoch in tqdm(range(1, cfg.epochs+1), desc="processing"):
    train_loss = my_trainer(
        model_s2s,
        train_loader,
        optimizer_s2s,
        loss_fn,
    )

    val_loss = my_evaluate(
        model_s2s,
        val_loader,
        loss_fn
    )

    s2s_early_stopping.step(
        val_loss,
        model_s2s
        )

    print(f"[Epoch {epoch}] train loss = {train_loss:.4f}")
    print(f"[Epoch {epoch}] val loss = {val_loss:.4f}")

processing:  10%|█         | 1/10 [29:12<4:22:52, 1752.52s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 1] train loss = 4.2094
[Epoch 1] val loss = 3.7189


processing:  20%|██        | 2/10 [48:15<3:05:51, 1393.90s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 2] train loss = 3.5309
[Epoch 2] val loss = 3.4134


processing:  30%|███       | 3/10 [1:07:54<2:31:11, 1295.87s/it]

...

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 8] train loss = 2.9196
[Epoch 8] val loss = 2.9898


processing:  90%|█████████ | 9/10 [3:54:34<34:29, 2069.33s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 9] train loss = 2.8840
[Epoch 9] val loss = 2.9717


processing: 100%|██████████| 10/10 [4:20:12<00:00, 1561.22s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 10] train loss = 2.8528
[Epoch 10] val loss = 2.9487


In [ ]:
# s2s 모델 추가 훈련 (5 에포크)
# 첫 10 에포크에서 조기종료가 되지 않았다는 점에서 개선 여지가 있고
# BLEU 점수가 아직 충분하지 않음
for epoch in tqdm(range(1, 6), desc="processing"):
    train_loss = my_trainer(
        model_s2s,
        train_loader,
        optimizer_s2s,
        loss_fn,
    )

    val_loss = my_evaluate(
        model_s2s,
        val_loader,
        loss_fn
    )

    s2s_early_stopping.step(
        val_loss,
        model_s2s
        )

    print(f"[Epoch {epoch}] train loss = {train_loss:.4f}")
    print(f"[Epoch {epoch}] val loss = {val_loss:.4f}")

processing:  20%|██        | 1/5 [56:29<3:45:59, 3389.98s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 1] train loss = 2.8291
[Epoch 1] val loss = 2.9377


processing:  40%|████      | 2/5 [1:29:23<2:07:49, 2556.50s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 2] train loss = 2.8044
[Epoch 2] val loss = 2.9132


processing:  60%|██████    | 3/5 [2:13:08<1:26:16, 2588.07s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 3] train loss = 2.7856
[Epoch 3] val loss = 2.8991


processing:  80%|████████  | 4/5 [3:04:49<46:30, 2790.42s/it]  

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 4] train loss = 2.7671
[Epoch 4] val loss = 2.8862


processing: 100%|██████████| 5/5 [4:00:01<00:00, 2880.36s/it]

Validation loss improved. Best model saved to models/s2s_model.pt
[Epoch 5] train loss = 2.7520
[Epoch 5] val loss = 2.8748


In [ ]:
ckpt_s2s = torch.load(cfg.model_dir / "s2s_model.pt", map_location=cfg.device)
model_s2s.load_state_dict(ckpt_s2s["model_state"])

<All keys matched successfully>

In [ ]:
# attn s2s 모델 훈련 (5 에포크)
for epoch in tqdm(range(1, cfg.epochs+1), desc="processing"):
    train_loss = my_trainer(
        model_attn,
        train_loader,
        optimizer_attn,
        loss_fn,
    )

    val_loss = my_evaluate(
        model_attn,
        val_loader,
        loss_fn
    )

    attn_early_stopping.step(
    val_loss,
    model_attn
    )

    print(f"[Epoch {epoch}] train loss = {train_loss:.4f}")
    print(f"[Epoch {epoch}] val loss = {val_loss:.4f}")

processing:  10%|█         | 1/10 [38:40<5:48:02, 2320.28s/it]

Validation loss improved. Best model saved to models/attn_model.pt
[Epoch 1] train loss = 3.5154
[Epoch 1] val loss = 3.0478


processing:  20%|██        | 2/10 [1:54:38<8:04:55, 3637.00s/it]

Validation loss improved. Best model saved to models/attn_model.pt
[Epoch 2] train loss = 2.9020
[Epoch 2] val loss = 2.8666


processing:  30%|███       | 3/10 [3:12:18<7:58:47, 4103.90s/it]

...

Validation loss improved. Best model saved to models/attn_model.pt
[Epoch 8] train loss = 2.4568
[Epoch 8] val loss = 2.6480


processing:  90%|█████████ | 9/10 [10:05:42<1:13:02, 4382.13s/it]

Validation loss improved. Best model saved to models/attn_model.pt
[Epoch 9] train loss = 2.4326
[Epoch 9] val loss = 2.6306


processing: 100%|██████████| 10/10 [11:23:28<00:00, 4100.81s/it] 

Validation loss improved. Best model saved to models/attn_model.pt
[Epoch 10] train loss = 2.4110
[Epoch 10] val loss = 2.6170


In [ ]:
# attn s2s 모델 추가 훈련
# 첫 10 에포크에서 조기종료가 되지 않았다는 점에서 개선 여지가 있고
# BLEU 점수가 아직 충분하지 않음
for epoch in tqdm(range(1, 6), desc="processing"):
    train_loss = my_trainer(
        model_attn,
        train_loader,
        optimizer_attn,
        loss_fn,
    )

    val_loss = my_evaluate(
        model_attn,
        val_loader,
        loss_fn
    )

    attn_early_stopping.step(
    val_loss,
    model_attn
    )

    print(f"[Epoch {epoch}] train loss = {train_loss:.4f}")
    print(f"[Epoch {epoch}] val loss = {val_loss:.4f}")

processing:  20%|██        | 1/5 [41:32<2:46:10, 2492.66s/it]

EarlyStopping counter: 1 / 3
[Epoch 1] train loss = 2.3223
[Epoch 1] val loss = 2.5823


processing:  40%|████      | 2/5 [1:46:07<2:45:17, 3305.85s/it]

EarlyStopping counter: 2 / 3
[Epoch 2] train loss = 2.3112
[Epoch 2] val loss = 2.5885


processing:  60%|██████    | 3/5 [2:57:53<2:05:24, 3762.39s/it]

EarlyStopping counter: 3 / 3
[Epoch 3] train loss = 2.3028
[Epoch 3] val loss = 2.5753


processing:  80%|████████  | 4/5 [3:35:59<52:59, 3179.51s/it]  

EarlyStopping counter: 4 / 3
[Epoch 4] train loss = 2.2910
[Epoch 4] val loss = 2.5769


processing: 100%|██████████| 5/5 [4:22:49<00:00, 3153.96s/it]

Validation loss improved. Best model saved to models/attn_model.pt
[Epoch 5] train loss = 2.2827
[Epoch 5] val loss = 2.5703


In [ ]:
ckpt_attn = torch.load(cfg.model_dir / "attn_model.pt", map_location=cfg.device)
model_attn.load_state_dict(ckpt_attn["model_state"])

<All keys matched successfully>

# 7. 추론 및 평가

In [ ]:
# 추론 함수 (Greedy)
@torch.no_grad()
def translate(model, src_text, src_tok=ko_tok, tgt_tok=en_tok, device=cfg.device, max_len: int=cfg.max_len):
    """
    추론 함수

    args:
        - model   : 사용 모델
        - src_text: 소스 언어 텍스트
        - src_tok : 소스 언어 토크나이저
        - tgt_tok : 타겟 언어 토크나이저
        - device  : 디바이스
        - max_len : 최대 길이

    returns:
        - translation: 타겟 언어 번역
    """
    model.eval()

    # 소스 문장 토큰화
    src_ids = src_tok.encode(src_text, max_len)
    src = torch.tensor(src_ids).unsqueeze(0).to(device)

    # 인코더 출력 결과
    enc_out, hidden = model.enc(src)

    # 어텐션용 마스크
    mask = (src != src_tok.pad_id)

    # 디코더용 첫 번째 토큰
    x = torch.tensor([tgt_tok.bos_id], device=device)
    out = []

    # 일반 seq2seq인가 attention seq2seq인가에 따라 분기
    for _ in range(max_len):
        if isinstance(model, Seq2SeqWithAttention):
            logits, hidden = model.dec(x, hidden, enc_out, mask)
        else:
            logits, hidden = model.dec(x, hidden)

        x = logits.argmax(1)
        if x.item() == tgt_tok.eos_id:
            break
        out.append(x.item())

    return tgt_tok.decode(out)

In [ ]:
# 추론 테스트
src = "날 좋아해?"

# seq2seq
pred_s = translate(
    model_s2s,
    src,
)

# attn seq2seq
pred_a = translate(
    model_attn,
    src,
)

print("KO:", src)
print("S2S EN:", pred_s)
print("S2S with Attn EN:", pred_a)

KO: 날 좋아해?
S2S EN: I like to do it like this?
S2S with Attn EN: You like me?


In [ ]:
# 검증 데이터셋 샘플 번역
s_src = val_normalized[15]["ko"]

# seq2seq
p_pred_s = translate(
    model_s2s,
    s_src,
)

# attn seq2seq
p_pred_a = translate(
    model_attn,
    s_src,
)

print("KO:", s_src)
print("S2S EN:", p_pred_s)
print("S2S with Attn EN:", p_pred_a)

KO: 오픈과 동시에 이벤트를 진행하려고 합니다.
S2S EN: We are planning to open a event and opening the same time.
S2S with Attn EN: We're going to hold an event at the same time.


In [ ]:
def trans_with_refer(model, loader, src_tok=ko_tok, tgt_tok=en_tok, device=cfg.device):
    model.eval()
    hyps = []
    refs = []

    with torch.no_grad():
        for src, tgt in loader:
            src = src.to(device)

            for i in range(src.size(0)):
                # src 토큰 -> 문자열
                src_text = src_tok.decode(src[i].tolist())

                # 모델 추론
                pred_text = translate(
                    model,
                    src_text,
                    src_tok=src_tok,
                    tgt_tok=tgt_tok,
                    device=device
                )

                # 정답 텍스트
                ref_text = tgt_tok.decode(tgt[i].tolist())

                hyps.append(pred_text)
                refs.append(ref_text)

    return hyps, refs


In [ ]:
# BLEU 평가(s2s)
hyps, refs = trans_with_refer(
    model_s2s,
    val_loader,
)

s2s_bleu = sacrebleu.corpus_bleu(hyps, [refs])

print(f"BLEU score (s2s): {s2s_bleu.score:.2f}")

BLEU score (s2s): 16.87


In [ ]:
# BLEU 평가(s2s attn)
h_hyps, r_refs = trans_with_refer(
    model_attn,
    val_loader,
)

attn_bleu = sacrebleu.corpus_bleu(h_hyps, [r_refs])

print(f"BLEU score (s2s with attn): {attn_bleu.score:.2f}")

BLEU score (s2s with attn): 21.81


## 7.1 정량적 평가

### BLEU 평가
- Seq2Seq (10 에포크): 14.01
- Seq2Seq (15 에포크): 16.87
- Seq2Seq with Attention (10 에포크): 19.38
- Seq2Seq with Attention (15 에포크): 21.81